### Transformaciones de datos

In [1]:
# Import libreries
import pandas as pd
from pdfquery import PDFQuery
import os
import pdfplumber
import numpy as np

In [2]:
# Ruta directorio
root_dir = os.getcwd()
print(root_dir)

c:\Users\rfigu\Documents\Python Scripts\WebScrapp_pdf


In [3]:
# Definimos las Columnas de la pagina 1 del pdf Resumen de Tasas
colPag1= ['Banco', 'MinMax', 'TC_ML_Nominal', 'TC_ML_Efectiva', 'TC_ME_Nominal', 'TC_ME_Efectiva', 
          'Micro_ML_Nominal', 'Micro_ML_Efectiva','Micro_ME_Nominal','Micro_ME_Efectiva', 
          'VivAFD_ML_Nominal','VivAFD_ML_Efectiva','VivAFD_ME_Nominal','VivAFD_ME_Efectiva'
         ,'VivPro_ML_Nominal','VivPro_ML_Efectiva','VivPro_ME_Nominal','VivPro_ME_Efectiva']

# Definimos las Columnas de la pagina 1 del pdf Resumen de Tasas
colPag2= ['Banco', 'MinMax', 'PCom_ML_Nominal', 'PCom_ML_Efectiva', 'PCom_ME_Nominal', 'PCom_ME_Efectiva', 
          'PDes_ML_Nominal','PDes_ML_Efectiva','PDes_ME_Nominal','PDes_ME_Efectiva',
          'PPer_ML_Nominal','PPer_ML_Efectiva','PPer_ME_Nominal','PPer_ME_Efectiva'
         ,'Sobre_ML_Nominal','Sobre_ML_Efectiva','Sobre_ME_Nominal','Sobre_ME_Efectiva']

# Definimos las Columnas de la pagina 1 del pdf Resumen de Tasas
colPag3= ['Banco', 'MinMax', 'AH_ML_Nominal', 'AH_ML_Efectiva', 'AH_ME_Nominal', 'AH_ME_Efectiva', 
          'Cda180_ML_Nominal','Cda180_ML_Efectiva','Cda180_ME_Nominal','Cda180_ME_Efectiva',
          'Cda365_ML_Nominal','Cda365_ML_Efectiva','Cda365_ME_Nominal','Cda365_ME_Efectiva'
         ,'CDA_ML_Nominal','CDA_ML_Efectiva','CDA_ME_Nominal','CDA_ME_Efectiva']

In [ ]:
lstArch = []
for dirpath, dirnames, filenames in os.walk(root_dir):
    for filename in filenames:
        if filename.endswith('.pdf'):
            # Crear cadena ruta archivo
            file_paths = os.path.join(dirpath, filename)
            lstArch.append(filename)
            print(file_paths)
            # Extraer fecha de cierre del pdf
            # pdf = PDFQuery('descargas\BANCOS012024.pdf')
            pdf = PDFQuery(file_paths)
            pdf.load()
            ele_texto = pdf.pq('LTTextLineHorizontal:contains("RESUMEN DE TASAS")')
            textoTit = [t.text for t in ele_texto]

            ini = str(textoTit).find("MES:")
            fin = len(str(textoTit[1]))
            cadenaFch = str(textoTit[1])
            fch = cadenaFch[ini+3:fin-1]

            # print(cadenaFch[ini+3:fin-1]) -------------------------------
            first_page = pdf.pages[0]
            second_page = pdf.pages[1]
            third_page = pdf.pages[2]

            datatable1 = first_page.extract_tables()
            datatable2 = second_page.extract_tables()
            datatable3 = third_page.extract_tables()

            df_pag1 = pd.DataFrame(datatable1)
            df_pag2 = pd.DataFrame(second_page.extract_tables())
            df_pag3 = pd.DataFrame(third_page.extract_tables())

            # Creamos un arrays de listas
            vector1 = df_pag1.iloc[0].values
            vector2 = df_pag2.iloc[0].values
            vector3 = df_pag3.iloc[0].values

            # Transformamos el vector en lista
            lstvector1 = list(vector1)
            df_tasas1 = pd.DataFrame(lstvector1, columns=colPag1)

            lstvector2 = list(vector2)
            df_tasas2 = pd.DataFrame(lstvector2, columns=colPag2)

            lstvector3 = list(vector3)
            df_tasas3 = pd.DataFrame(lstvector3, columns=colPag3)

            # Eliminar primeras filas pag 1
            df_tasas1.drop(df_tasas1.index[0:4], inplace=True)
            # Eliminar primeras filas pag 2
            df_tasas2.drop(df_tasas2.index[0:4], inplace=True)
            # Eliminar primeras filas pag 3
            df_tasas3.drop(df_tasas3.index[0:4], inplace=True)
            # Reindexar el indice
            df_tasas1.reset_index(drop=True, inplace=True)
            df_tasas2.reset_index(drop=True, inplace=True)
            df_tasas3.reset_index(drop=True, inplace=True)
            # Agregando FechaCierre
            df_tasas1['FechaCierre']=fch
            df_tasas2['FechaCierre']=fch
            df_tasas3['FechaCierre']=fch

            

In [4]:
# Extraer fecha de cierre del pdf
pdf = PDFQuery('descargas\BANCOS012024.pdf')
pdf.load()
ele_texto = pdf.pq('LTTextLineHorizontal:contains("RESUMEN DE TASAS")')
textoTit = [t.text for t in ele_texto]

ini = str(textoTit).find("MES:")
fin = len(str(textoTit[1]))
cadenaFch = str(textoTit[1])

fch = cadenaFch[ini+3:fin-1]

print(fch)

Enero/2024


In [5]:
with pdfplumber.open('descargas\BANCOS012024.pdf') as pdf:

    first_page = pdf.pages[0]
    second_page = pdf.pages[1]
    third_page = pdf.pages[2]
    #  
    datatable1 = first_page.extract_tables()
    datatable2 = second_page.extract_tables()
    datatable3 = third_page.extract_tables()
    #
    df_pag1 = pd.DataFrame(datatable1)
    df_pag2 = pd.DataFrame(second_page.extract_tables())
    df_pag3 = pd.DataFrame(third_page.extract_tables())
    # Creamos un arrays de listas
    vector1 = df_pag1.iloc[0].values
    vector2 = df_pag2.iloc[0].values
    vector3 = df_pag3.iloc[0].values
    # Transformamos el vector en lista
    lstvector1 = list(vector1)
    df_tasas1 = pd.DataFrame(lstvector1, columns=colPag1)

    lstvector2 = list(vector2)
    df_tasas2 = pd.DataFrame(lstvector2, columns=colPag2)

    lstvector3 = list(vector3)
    df_tasas3 = pd.DataFrame(lstvector3, columns=colPag3)

    # Eliminar primeras filas pag 1
    df_tasas1.drop(df_tasas1.index[0:4], inplace=True)
    # Eliminar primeras filas pag 2
    df_tasas2.drop(df_tasas2.index[0:4], inplace=True)
    # Eliminar primeras filas pag 3
    df_tasas3.drop(df_tasas3.index[0:4], inplace=True)
    # Reindexar el indice
    df_tasas1.reset_index(drop=True, inplace=True)
    df_tasas2.reset_index(drop=True, inplace=True)
    df_tasas3.reset_index(drop=True, inplace=True)
    # Agregando FechaCierre
    df_tasas1['FechaCierre']=fch
    df_tasas2['FechaCierre']=fch
    df_tasas3['FechaCierre']=fch

In [6]:
df_tasas1.head(3)

,Banco,MinMax,TC_ML_Nominal,TC_ML_Efectiva,TC_ME_Nominal,TC_ME_Efectiva,Micro_ML_Nominal,Micro_ML_Efectiva,Micro_ME_Nominal,Micro_ME_Efectiva,VivAFD_ML_Nominal,VivAFD_ML_Efectiva,VivAFD_ME_Nominal,VivAFD_ME_Efectiva,VivPro_ML_Nominal,VivPro_ML_Efectiva,VivPro_ME_Nominal,VivPro_ME_Efectiva,FechaCierre
0,Banco Nacional de Fomento,Mínima,"1 4,00","1 4,94",-,-,"9 ,00","9 ,31","6 ,00","6 ,17","7 ,90","8 ,19",-,-,"7 ,95","8 ,25",-,-,Enero/2024
1,None,Máxima,"1 4,00","1 4,94",-,-,"1 2,00","1 2,55","8 ,75","9 ,11","1 1,40","1 2,02",-,-,"1 1,00","1 1,57",-,-,Enero/2024
2,Banco de la Nación Argentina,Mínima,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,Enero/2024


In [7]:
# Seleccionar columnas que comienzan con 'M'
col_MEyML = [col for col in df_tasas1.columns if '_M' in col]
# Desagrupar DataFrame
df_unpivot1 = pd.melt(df_tasas1, id_vars=['FechaCierre', 'Banco', 'MinMax'], value_vars= col_MEyML, var_name='TipoTasa')

# Ver DataFrame desagrupado
df_unpivot1.head()

,FechaCierre,Banco,MinMax,TipoTasa,value
0,Enero/2024,Banco Nacional de Fomento,Mínima,TC_ML_Nominal,"1 4,00"
1,Enero/2024,None,Máxima,TC_ML_Nominal,"1 4,00"
2,Enero/2024,Banco de la Nación Argentina,Mínima,TC_ML_Nominal,-
3,Enero/2024,None,Máxima,TC_ML_Nominal,-
4,Enero/2024,Banco GNB Paraguay S.A.,Mínima,TC_ML_Nominal,"1 5,65"


In [8]:
# Reemplazamos la coma con el punto
df_unpivot1.replace(',', '.', regex=True, inplace=True)
# eliminarmos espacios en blanco
df_unpivot1['value'] = df_unpivot1['value'].replace(' ', '', regex=True)

# Eliminar filas que contienen el carácter "-"
print(df_unpivot1.shape)
df_f1 = df_unpivot1[~df_unpivot1['value'].str.contains('-')].copy()

# Ver DataFrame resultante
print(df_f1.shape)

# Cambiar tipo de datos
df_f1['value'] = df_f1['value'].astype(float)
print('tipo de dato cambiado')

(512, 5)
(260, 5)
tipo de dato cambiado


In [9]:
df_f1.head()

,FechaCierre,Banco,MinMax,TipoTasa,value
0,Enero/2024,Banco Nacional de Fomento,Mínima,TC_ML_Nominal,14.00
1,Enero/2024,None,Máxima,TC_ML_Nominal,14.00
4,Enero/2024,Banco GNB Paraguay S.A.,Mínima,TC_ML_Nominal,15.65
5,Enero/2024,None,Máxima,TC_ML_Nominal,15.65
10,Enero/2024,Sudameris Bank S.A.E.C.A.,Mínima,TC_ML_Nominal,15.65


In [ ]:
# import pdfplumber
# import numpy as np

with pdfplumber.open('descargas\BANCOS012024.pdf') as pdf:
    first_page = pdf.pages[0]
    second_page = pdf.pages[1]
    third_page = pdf.pages[2]

    datatable1 = first_page.extract_tables()
    datatable2 = second_page.extract_tables()
    datatable3 = third_page.extract_tables()


    df_pag1 = pd.DataFrame(datatable1)
    df_pag2 = pd.DataFrame(second_page.extract_tables())
    df_pag3 = pd.DataFrame(third_page.extract_tables())


In [ ]:
# Creamos un arrays de listas
vector1 = df_pag1.iloc[0].values
vector2 = df_pag2.iloc[0].values
vector3 = df_pag3.iloc[0].values

# vector1

In [ ]:
# Transformamos el vector en lista
lstvector1 = list(vector1)
df_tasas1 = pd.DataFrame(lstvector1, columns=colPag1)

lstvector2 = list(vector2)
df_tasas2 = pd.DataFrame(lstvector2, columns=colPag2)

lstvector3 = list(vector3)
df_tasas3 = pd.DataFrame(lstvector3, columns=colPag3)

# df_tasas1.head()

In [ ]:
# Eliminar primeras filas pag 1
df_tasas1.drop(df_tasas1.index[0:4], inplace=True)

print('Filas, Col: '+str(df_tasas1.shape))
df_tasas1.head()

In [ ]:
# Eliminar primeras filas pag 2
df_tasas2.drop(df_tasas2.index[0:4], inplace=True)

print('Filas, Col: '+str(df_tasas2.shape))
df_tasas2.head()

In [ ]:
# Eliminar primeras filas pag 3
df_tasas3.drop(df_tasas3.index[0:4], inplace=True)

print('Filas, Col: '+str(df_tasas3.shape))
df_tasas3.head()

In [ ]:
df_tasas1.reset_index(drop=True, inplace=True)
df_tasas2.reset_index(drop=True, inplace=True)
df_tasas3.reset_index(drop=True, inplace=True)


In [ ]:
df_tasas1['FechaCierre']=fch
df_tasas1.head()

In [ ]:
df_tasas2['FechaCierre']=fch
df_tasas2.head(2)

In [ ]:
df_tasas3['FechaCierre']=fch
df_tasas3.head(2)